<a href="https://colab.research.google.com/github/Tara2712/Informatika/blob/main/Procesiranje_podatkov/dokon%C4%8Dna_predpriprava_podatkov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pd

In [101]:
df = pd.read_csv('opis_cleaned.csv', index_col='SR')

In [102]:
df.shape

(28353, 6)

In [103]:
# df.head()

In [104]:
df.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,900
OPIS,2499
DOLGI_OPIS_X,10210


In [105]:
stolpci_za_preverit = ["OPIS", "DOLGI_OPIS_X"]

num_both_null = df[(df['OPIS'].isna()) & (df['DOLGI_OPIS_X'].isna())].shape[0] #& (df['WORKLOGID'].isna())
print(f"Number of rows where both 'OPIS' and 'DOLGI_OPIS_X' are null: {num_both_null}")

Number of rows where both 'OPIS' and 'DOLGI_OPIS_X' are null: 1297


In [106]:
df_pociscen = df.dropna(subset=stolpci_za_preverit, how="all")

In [107]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,8913


In [108]:
def drop_empty_resitev(df):
    pogoj = ((df["OPIS"] == "Rešitev") | (df["OPIS"] == "Potrjujem rešitev") | (df["OPIS"] == "Zavračam rešitev")) & (df["DOLGI_OPIS_X"].isnull() | (df["DOLGI_OPIS_X"].str.strip() == ""))
    df_cleaned = df[~pogoj].copy()  # ~ znegira pogoj
    return df_cleaned

In [109]:
df_pociscen = drop_empty_resitev(df_pociscen)

In [110]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,518


In [111]:
import re
import numpy as np
def clean_and_null(text):
    if not isinstance(text, str):
        return np.nan
    cleaned = re.sub(r'img\s+[^>]*?src="data:image[^"]+"', '', text, flags=re.IGNORECASE).strip()
    return cleaned if cleaned else np.nan

df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].apply(clean_and_null)
# df_pociscen.iloc[87:91]


def odstrani_dodane_slike_v_besedilu(text):
    if not isinstance(text, str):
        return text

    return re.sub(r'img\s+[^>]*?src="data:image[^"]*$', '', text, flags=re.IGNORECASE)

df_pociscen["DOLGI_OPIS_X"] = df_pociscen["DOLGI_OPIS_X"].apply(odstrani_dodane_slike_v_besedilu) #.fillna("")

In [112]:
df_pociscen["DOLGI_OPIS_X"].replace('', np.nan, inplace=True)

<ipython-input-112-8087d3f06975>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_pociscen["DOLGI_OPIS_X"].replace('', np.nan, inplace=True)


In [113]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,638


In [114]:
# df_pociscen['OPIS'].value_counts()

In [115]:
# df_pociscen.head()

In [116]:
def drop_empty_zavracam_resitev(df):
    pogoj = ((df["OPIS"] == "Zavračam rešitev") | (df["OPIS"] == "Zapiramo zahtevek")| (df["OPIS"] == "Zahtevek zapiramo") | (df["OPIS"] == "Urejeno")) & (df["DOLGI_OPIS_X"].isnull() | (df["DOLGI_OPIS_X"].str.strip() == "") )
    df_cleaned = df[~pogoj].copy()
    return df_cleaned

In [117]:
df_pociscen = drop_empty_zavracam_resitev(df_pociscen)

In [118]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,629


In [119]:
df_pociscen.shape

(18652, 6)

In [120]:
df_pociscen["OPIS"].value_counts()

,count
OPIS,
Rešitev,10978
Komentar IN,279
Komentar INF,252
Odgovor,217
Zahtevek sprejet,172
...,...
Zapis pri fakturi v delovnem toku,1
Odgovor informatika,1
sprejeto v reševanje,1


In [121]:
# df_pociscen.to_csv('2proba.csv', index=True)

In [122]:
# df_pociscen = df_pociscen['DOLGI_OPIS_X'].dropna()

In [123]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1202
DOLGI_OPIS_X,629


In [124]:
df_pociscen = df_pociscen.dropna(subset=['DOLGI_OPIS_X'])
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,1186
DOLGI_OPIS_X,0


In [125]:
df_pociscen.shape

(18023, 6)

In [126]:
from sklearn.impute import SimpleImputer
import numpy as np

In [127]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df_pociscen["OPIS"] = imputer.fit_transform(df_pociscen[["OPIS"]]).ravel()

In [128]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [129]:
df_pociscen.shape

(18023, 6)

In [130]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0


In [131]:
df_pociscen.to_csv("df_no_nan_img.csv", index=True)

In [132]:
df_pociscen.isna().sum()

,0
NAZIV_SR,0
DATUM_NASTANKA_SR,0
STATUS,0
WORKLOGID,0
OPIS,0
DOLGI_OPIS_X,0
